In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
#extracting data from the pdf file
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="**/*.pdf",  # Correct keyword
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

In [5]:
extracted_data=load_pdf_file(data='Data/')

In [6]:
# split the data into text chunks\
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return(text_chunks)


In [7]:
text_chunks= text_split(extracted_data)
print('length of text chunks' ,len(text_chunks))

length of text chunks 1733


In [8]:
#text_chunks


In [9]:
from langchain.embeddings import HuggingFaceEmbeddings

In [10]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return(embeddings)


In [11]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")


In [12]:
embeddings=download_hugging_face_embeddings()

C:\Users\Muralikrishna\AppData\Local\Temp\ipykernel_10052\1547537371.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [13]:
query_result = embeddings.embed_query('hello world')
print('Length',len(query_result))

Length 384


In [14]:
#query_result

In [15]:
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [16]:
import os
PINECONE_API_KEY = os.environ.get('PINECCONE_API_KEY')

In [18]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = 'medicalchatbot-3'


pc.create_index(
    name=index_name,
    dimension=384,  
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

{
    "name": "medicalchatbot-3",
    "metric": "cosine",
    "host": "medicalchatbot-3-tkm92y1.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [19]:
#embedding each vectoe chunk and uploading it into pinecone vector storage
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding = embeddings


)

In [20]:
#loading existing index from pinecone
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding = embeddings
)

In [21]:
retriever = docsearch.as_retriever(search_type = "similarity", search_kwargs={"k":3} )


In [22]:
retrieverdocs = retriever.invoke("what is asthama and how to cure it")
retrieverdocs

[Document(id='07ca2d47-1ecf-4f07-b6d8-ba627f8d868d', metadata={'creationdate': '2025-01-30T11:41:24+00:00', 'creator': 'Chromium', 'moddate': '2025-01-30T11:41:24+00:00', 'page': 102.0, 'page_label': '103', 'producer': 'Skia/PDF m91', 'source': 'Data\\medical book.pdf', 'total_pages': 409.0}, page_content='Page 103 / 409\nAsthma\n\xa0\nLast updated: June 2023\n\xa0\n\xa0\n\xa0\nAsthma is a chronic inﬂammatory disorder of the airways associated with airway hyperresponsiveness that leads to\nrecurrent episodes of wheezing, breathlessness, chest tightness and coughing. These episodes are usually associated\nwith airﬂow obstruction within the lung, often reversible, either spontaneously or with treatment.\xa0Factors that precipitate/aggravate asthma include: allergens, infection, exercise, drugs (aspirin), tobacco,'),
 Document(id='c76baa2f-44a4-4264-a76b-8c930f71775f', metadata={'creationdate': '2025-01-30T11:41:24+00:00', 'creator': 'Chromium', 'moddate': '2025-01-30T11:41:24+00:00', 'pa

In [23]:
from dotenv import load_dotenv
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [24]:
import os
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [25]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.4,
    max_tokens = 500,
    model_name="gpt-3.5-turbo",
    openai_api_key="OPENAI_API_KEY"
)


C:\Users\Muralikrishna\AppData\Local\Temp\ipykernel_10052\1448479436.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [26]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    " you are an assistant for question-answering tasks."
    "use the folowing pieces of retrieved context to answer the question"
    "if you dont know the answer, say that you dont know. "
    "use three sentence maximum and kepp the answer concise"
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]

)

In [27]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain )

In [28]:
responce = rag_chain.invoke({"input": "what is asthama"})
print(responce["answer"])

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: OPENAI_A**_KEY. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}